In [7]:
#permutations for keys
p10_seq = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
p8_seq = (6, 3, 7, 4, 8, 5, 10, 9)

#permutations for text
ip_seq = (2, 6, 3, 1, 4, 8, 5, 7)
inv_ip_seq = (4, 1, 3, 5, 7, 2, 8, 6)

#permutation to expand 4 bit to 8 bit
ep_seq = (4, 1, 2, 3, 2, 3, 4, 1)

#permutation for 4 bits
p4_seq = (2, 4, 3, 1)

#s boxes
s0_seq = [
            ["01", "00", "11", "10"],
            ["11", "10", "01", "00"],
            ["00", "10", "01", "11"],
            ["11", "01", "11", "10"]
         ]

s1_seq = [
            ["00", "01", "10", "11"],
            ["10", "00", "01", "11"],
            ["11", "00", "01", "00"],
            ["10", "01", "00", "11"]
         ]

In [8]:
def left_shift(s, bits):
    s = s[bits:] + s[:bits]
    
    return s

In [9]:
def permute_and_generate(ip, seq):
    s=""
    
    for val in seq:
        s += ip[val-1]
        
    return s

In [10]:
def generate_keys(key):
    
    p10 = permute_and_generate(key, p10_seq)
    
    print("P10(Key) : ", p10)
    
    p10_left = p10[0:5]
    p10_right = p10[5:10]
    
    ls1_left = left_shift(p10_left, 1)
    ls1_right = left_shift(p10_right, 1)
    
    print("Shift1(P10(Key)) : ", ls1_left + ls1_right)
    
    k1 = permute_and_generate(ls1_left + ls1_right, p8_seq)
    
    print("k1 : ", k1)
    
    ls2_left = left_shift(ls1_left, 2)
    ls2_right = left_shift(ls1_right, 2)
    
    print("Shift3(P10(Key)) : ", ls2_left + ls2_right)
    
    k2 = permute_and_generate(ls2_left + ls2_right, p8_seq)
    
    
    print("k2 : ", k2)
    
    return k1, k2

In [11]:
def find_xor(s1, s2):
    xor = ""
    
    for i in range(len(s1)):
        if s1[i] == s2[i]:
            xor += "0"
        else:
            xor += "1"
            
    return xor

In [12]:
def find_s0_s1(xor_half, lookup_table):
    r = (int(xor_half[0]) * 2) + int(xor_half[3])
    c = (int(xor_half[1]) * 2) + int(xor_half[2])
    
    return lookup_table[r][c]

In [13]:
def round_encrypt(ip, key):
    
    expanded_per = permute_and_generate(ip, ep_seq)
    
    print("EP(IP) : ", expanded_per)
    
    expanded_per_xor = find_xor(expanded_per, key)
    
    print("EP(IP) XOR key : ", expanded_per_xor)
        
    left_ep = expanded_per_xor[:4]
    right_ep = expanded_per_xor[4:]
    
    s0 = find_s0_s1(left_ep, s0_seq)
    s1 = find_s0_s1(right_ep, s1_seq)
    
    print("s0 : ", s0)
    print("s1 : ", s1)
    
    p4 = permute_and_generate(s0 + s1, p4_seq)
    
    return p4

In [14]:
def encrypt_decrypt(ip, k1, k2):
    
    ip_per = permute_and_generate(ip, ip_seq)
    
    print("IP(IP) : ", ip_per)
    
    left_ip_per = ip_per[:4]
    right_ip_per = ip_per[4:]
    
    # round 1
    r1_output = round_encrypt(right_ip_per, k1)
    
    print("round1(R, k1) : ", r1_output)
    
    r1_output = find_xor(r1_output, left_ip_per)
    
    print("L XOR round1(r1_output, k1) : ", r1_output)
    
    # round 2
    r2_output = round_encrypt(r1_output, k2)
    print("round2(r1_output, k2) : ", r2_output)
    
    r2_output = find_xor(r2_output, right_ip_per)
    print("R XOR round2(r2_output, k2) : ", r2_output)
    
    ip_inv = permute_and_generate(r2_output + r1_output, inv_ip_seq)
    
    return ip_inv

In [20]:
k1, k2 = generate_keys("1100011110")

P10(Key) :  0011001111
Shift1(P10(Key)) :  0110011110
k1 :  11101001
Shift3(P10(Key)) :  1000111011
k2 :  10100111


In [21]:
plaintext = "00101000"

print("Plaintext : ", plaintext, "\n\n")

ciphertext = encrypt_decrypt(plaintext, k1, k2)

print("\nCiphertext : ", ciphertext, "\n\n")

decryptedtext = encrypt_decrypt(ciphertext, k2, k1)

print("\nDecrypted text : ", decryptedtext)

Plaintext :  00101000 


IP(IP) :  00100010
EP(IP) :  00010100
EP(IP) XOR key :  11111101
s0 :  10
s1 :  00
round1(R, k1) :  0001
L XOR round1(r1_output, k1) :  0011
EP(IP) :  10010110
EP(IP) XOR key :  00110001
s0 :  10
s1 :  10
round2(r1_output, k2) :  0011
R XOR round2(r2_output, k2) :  0001

Ciphertext :  10001010 


IP(IP) :  00010011
EP(IP) :  10010110
EP(IP) XOR key :  00110001
s0 :  10
s1 :  10
round1(R, k1) :  0011
L XOR round1(r1_output, k1) :  0010
EP(IP) :  00010100
EP(IP) XOR key :  11111101
s0 :  10
s1 :  00
round2(r1_output, k2) :  0001
R XOR round2(r2_output, k2) :  0010

Decrypted text :  00101000
